In [1]:
# TaylorKAN + CNN
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


class TaylorLayer(nn.Module):
  def __init__(self, input_dim, out_dim, order, addbias=True):
    super(TaylorLayer, self).__init__()
    self.input_dim = input_dim
    self.out_dim = out_dim
    self.order = order
    self.addbias = addbias

    # Initialize Taylor coefficients
    self.coeffs = nn.Parameter(torch.randn(out_dim, input_dim, order) * 0.01)  
    if self.addbias:
      self.bias = nn.Parameter(torch.zeros(1, out_dim)) 

  def forward(self, x):
    shape = x.shape
    outshape = shape[0:-1] + (self.out_dim,)
    x = torch.reshape(x, (-1, self.input_dim)) 

    x_expanded = x.unsqueeze(1).expand(-1, self.out_dim, -1)

    # Compute and accumulate each term of the Taylor expansion
    y = torch.zeros((x.shape[0], self.out_dim), device=x.device)  

    for i in range(self.order):
      term = (x_expanded ** i) * self.coeffs[:, :, i]  
      y += term.sum(dim=-1) 

    if self.addbias:
      y += self.bias  

    y = torch.reshape(y, outshape)
    return y

class TaylorCNN(nn.Module):
  def __init__(self):
    super(TaylorCNN, self).__init__()
    self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
    self.pool1 = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
    self.pool2 = nn.MaxPool2d(2)
    self.taylorkan1 = TaylorLayer(32*7*7, 128, 2)
    self.taylorkan2 = TaylorLayer(128, 10, 2)

  def forward(self, x):
    x = F.selu(self.conv1(x))
    x = self.pool1(x)
    x = F.selu(self.conv2(x))
    x = self.pool2(x)
    x = x.view(x.size(0), -1)
    x = self.taylorkan1(x)
    x = self.taylorkan2(x)
    return x


# Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TaylorCNN().to(device)

# optimizer = optim.LBFGS(model.parameters(), lr=0.001)
# optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=1e-4, momentum=0.9)
optimizer = optim.RAdam(model.parameters(), lr=0.0001)



# Training
def train(model, device, train_loader, optimizer, epoch):
  model.train()
  for i, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = model(data.to(device))
    loss = nn.CrossEntropyLoss()(output, target.to(device))
    loss.backward()
    optimizer.step()
    if i % 10 == 0:
      print(f'Train Epoch: {epoch} [{i * len(data)}/{len(train_loader.dataset)} ({100. * i / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Evaluation
def evaluate(model, device, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += nn.CrossEntropyLoss()(output, target).item()
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

# Running
for epoch in range(0, 2):
  train(model, device, train_loader, optimizer, epoch)
evaluate(model, device, test_loader)

c:\Users\Muyuzhi\.conda\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [01:22<00:00, 120275.15it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 90122.48it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:13<00:00, 118485.01it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 130117.67it/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.312465
Train Epoch: 0 [640/60000 (1%)]	Loss: 2.309981
Train Epoch: 0 [1280/60000 (2%)]	Loss: 2.293256
Train Epoch: 0 [1920/60000 (3%)]	Loss: 2.294332
Train Epoch: 0 [2560/60000 (4%)]	Loss: 2.320176
Train Epoch: 0 [3200/60000 (5%)]	Loss: 2.287000
Train Epoch: 0 [3840/60000 (6%)]	Loss: 2.282419
Train Epoch: 0 [4480/60000 (7%)]	Loss: 2.275718
Train Epoch: 0 [5120/60000 (9%)]	Loss: 2.258961
Train Epoch: 0 [5760/60000 (10%)]	Loss: 2.260962
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.266090
Train Epoch: 0 [7040/60000 (12%)]	Loss: 2.257350
Train Epoch: 0 [7680/60000 (13%)]	Loss: 2.228045
Train Epoch: 0 [8320/60000 (14%)]	Loss: 2.184229
Train Epoch: 0 [8960/60000 (15%)]	Loss: 2.193384
Train Epoch: 0 [9600/60000 (16%)]	Loss: 2.193172
Train Epoch: 0 [10240/60000 (17%)]	Loss: 2.166692
Train Epoch: 0 [10880/60000 (18%)]	Loss: 2.092022
Train Epoch: 0 [11520/60000 (19%)]	Loss: 2.106154
Trai